In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

#visualize Augmentation from directory!
def looking_at_augmentation (data_generator, batchsize, path):
    im, label = next(data_generator)
    print(im)
    im = (im - np.min(im))/np.ptp(im) # to normalize all images --> matplotlib only takes pos. values between 0..1 / 0..255 
    print(im)
    imgs = list(im)
    labels = list(label)
    
    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=0.5)
    plt.suptitle('Augmented Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')

    for i, im  in enumerate(imgs[:batchsize]):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(im)
        fig.set_figheight(8)
        fig.set_figwidth(8)

    #fig.tight_layout()
    fig.savefig(base_path + '\\Augmented-Images.png', dpi=300)

#fast plot of training history
def plot_history(history, modelname, path):
    hist_df = pd.DataFrame(history.history)
    fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
    axs[0].plot(hist_df.val_categorical_accuracy, lw=5, label='Validation Accuracy')
    axs[0].plot(hist_df.categorical_accuracy, lw=5, label='Training Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].grid()
    axs[0].legend(loc=0)
    axs[1].plot(hist_df.val_loss, lw=5, label='Validation MLogLoss')
    axs[1].plot(hist_df.loss, lw=5, label='Training MLogLoss')
    axs[1].set_ylabel('MLogLoss')
    axs[1].set_xlabel('Epoch')
    axs[1].grid()
    axs[1].legend(loc=0)
    fig.savefig(path + '\History_{}.png' .format(modelname), dpi=300)
    hist_df.to_csv()
    plt.show();

from sklearn.metrics import roc_curve, roc_auc_score, auc#plotting the receiver operating characteristics --> evaluate performance cutting point vice
def plot_roc(label, predictions, modelname, path): #IDEA: set diffrent cutting point based on ROC for ensembling   
    roc_auc_score(label, predictions)
    print('The ROC-Score is: {}' .format(roc_auc_score))

    fpr_keras, tpr_keras, thresholds_keras = roc_curve(label, predictions)
    auc_keras = auc(fpr_keras, tpr_keras)
    #print(auc_keras)

    fig = plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve: {}' .format(auc_keras))
    plt.legend(loc='best')
    fig.savefig(path + '\ROC-Curve_{}.png' .format(modelname), dpi=300) #saving PLOT 
    plt.show()
    
from sklearn.metrics import confusion_matrix
import itertools
# Source: Scikit Learn website
# http://scikit-learn.org/stable/auto_examples/
# model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-
# selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
#plotting correctly classified images: https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python
def plot_correct(vals, y_pred, y_label, modelname, path):
    correct = np.where(y_pred==y_label)[0]
    print ("Found %d correct labels" % len(correct))


    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=0.5)
    plt.suptitle('Correct Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')

    for i, correct in enumerate(correct[:9]):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(vals[correct])
        ax.set_title("Predicted {}, Class {}".format(y_pred[correct], y_label[correct]), fontsize=10)
        fig.set_figheight(8)
        fig.set_figwidth(8)

    fig.savefig(path + '\Correct_Images_{}.png' .format(modelname), dpi=100) #saving PLOT 

#Plotting incorrectly classified
def plot_incorrect(vals, y_pred, y_label, modelname, path):
    incorrect = np.where(y_pred!=y_label)[0]
    print ("Found %d incorrect labels" % len(incorrect))


    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=1)
    plt.suptitle('Incorrect Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')

    for i, incorrect in enumerate(incorrect[:9]):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(vals[incorrect])
        ax.set_title("Predicted {}, Class {}".format(y_pred[incorrect], y_label[incorrect]), fontsize=10)
        fig.set_figheight(8)
        fig.set_figwidth(8)

    fig.savefig(path + '\Incorrect_Images_{}.png' .format(modelname), dpi=100) #saving PLOT 
    
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [ ]:
%matplotlib notebook
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import shutil
print(os.listdir('/home/Deep_Learner/work/local/Tiles_272/Validation'))

from glob import glob 
from skimage.io import imread
import gc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical # get categorical labels

import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from keras import backend
from keras import applications
from keras.models import Model
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, BatchNormalization, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint, Callback, LambdaCallback, CSVLogger
from keras.metrics import categorical_accuracy

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.5), # vertically flip 20% of all images
        sometimes(iaa.Affine(
            scale={"x": (0.9, 1.6), "y": (0.9, 1.6)}, #>20 will cut part of img
            translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, # >20% will also cut part of img
            rotate=(-10, 10), # 45/-45° -> works good with scale + translate to prevent cuts
            shear=(-5, 5), # shear by -16 to +16 degrees
            mode=ia.ALL 
        )),
        iaa.SomeOf((0, 4), [
                sometimes(iaa.Superpixels(p_replace=(0.3, 0.7), n_segments=(10, 100))), #superpixel-representation --> better basallamina representation 
                iaa.OneOf([
                    iaa.GaussianBlur((0, 0.2)), #small blur effects --> better representation
                    iaa.AverageBlur(k=(1, 3)), # k must be odd
                    iaa.MedianBlur(k=(1, 3)), # 
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), #cell wall represenation
                iaa.Emboss(alpha=(0, 0.8), strength=(0, 0.5)), #cell wall represenation
                #searching for edges or angles --> blobby mask --> better basallamina representation / nuclei
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.2, 0.4)), #detects edges --> cell wall,..
                    iaa.DirectedEdgeDetect(alpha=(0.2, 0.4), direction=(0.0, 1.0)), #direction will make edges from random directions 
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.2), # add gaussian noise to images
             iaa.OneOf([
                    iaa.Dropout((0.05, 0.3), per_channel=0.2), #rnd remove 5% in small pixels
                    iaa.CoarseDropout((0.05, 0.3), size_percent=(0.01, 0.02), per_channel=0.2),# rnd remove 3% in big pixels
                ]),
                iaa.Invert(0.01, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.3), # change brightness of images (by -10 to 10 of original value)
                #iaa.AddToHueAndSaturation((-0.1, 0.1)), # change hue and saturation
                #
                #either change the brightness of the whole image (sometimes per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.9, 1.2), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-1, 0),
                        first=iaa.Multiply((0.9, 1.1), per_channel=True),
                        second=iaa.ContrastNormalization((0.9, 1.1))
                    )
                ]),
                sometimes(iaa.ElasticTransformation(alpha=(0, 0.5), sigma=0.1)), #still not sure: move pixels locally around
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.03))), #still not sure:move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
                     random_order=True
        )
    ],
    random_order=True
)

In [ ]:
IMAGE_SIZE = 100
IMAGE_CHANNELS = 3

model_name = 'VGG_newtoplayer'

base_path = ('/home/Deep_Learner/work/local/Models/{}'.format(model_name))
if not os.path.exists(base_path):
    os.makedirs(base_path)

train_path = ('/home/Deep_Learner/work/local/Tiles_272/Validation')
val_path = ('/home/Deep_Learner/work/local/Tiles_272/Validation')

In [ ]:
train_batch_size_1 = 128
val_batch_size = 64

datagen_train = ImageDataGenerator(preprocessing_function=seq.augment_image,
                            rescale=1./255)

datagen_val = ImageDataGenerator(rescale=1./255)

datagen_no_aug = ImageDataGenerator(rescale=1./255)

train_gen_1 = datagen_train.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size_1,
                                        class_mode='categorical')

train_gen_noaug = datagen_train.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size_1,
                                        class_mode='categorical')

val_gen = datagen_val.flow_from_directory(val_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

num_train_samples = train_gen_1.samples
num_val_samples = val_gen.samples
print(num_train_samples)
print(num_val_samples)

In [ ]:
nb_train_samples = len(train_gen_noaug)
nb_validation_samples = len(val_gen)

batches = train_generator
val_batches = validation_generator
test_batches = test_generator
steps_per_epoch = int(np.ceil(batches.samples/batch_size))
validation_steps = int(np.ceil(val_batches.samples/batch_size))
test_steps = int(np.ceil(test_batches.samples/batch_size))

val_classes = val_batches.classes
trn_classes = batches.classes
test_classes = test_batches.classes
val_labels = to_categorical(val_classes)
trn_labels = to_categorical(trn_classes)
test_labels = to_categorical(test_classes)

In [ ]:
input_shape=IMAGE_SIZE, IMAGE_SIZE, 3
model_input = Input(shape=input_shape)

In [ ]:
base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_tensor=model_input, input_shape=input_shape)

In [ ]:
val_features = base_model.predict_generator(val_batches, validation_steps)

trn_features = base_model.predict_generator(batches, steps_per_epoch)

In [ ]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

save_array(model_path + 'train_base_model_xception_2_features.bc', trn_features)
save_array(model_path + 'valid_base_model_xception_2_features.bc', val_features)

In [ ]:
trn_features = load_array(model_path+'train_base_model_xception_2_features.bc')
val_features = load_array(model_path+'valid_base_model_xception_2_features.bc')

trn_features.shape

base_model.output_shape

In [ ]:
def top_model_tr():
    inputs = Input(shape=base_model.output_shape[1:])
    x = GlobalMaxPooling2D(name='1')(x)
    x = BatchNormalization(name='5')(out)
    x = Dropout(0.5, name='6')(x)
    x = Dense(512, activation='relu',name='7') (x)
    x = BatchNormalization(name='8')(x)
    x = Dropout(0.2, name='9')(x)
    x = Dense(256, activation='relu',name='10') (x)
    x = BatchNormalization(name='11')(x)
    
    predictions = (Dense(2, activation='softmax'))(x)
    model_VGG = Model(inputs=inputs, outputs=predictions)
    
    return top_model_tr


In [ ]:
top_model_ttr= top_model_tr()
top_model_ttr.summary()

In [ ]:
for layer in top_model_ttr.layers:
    print(layer.name)
for layer in top_model_ttr.layers:
    print(layer, layer.trainable)

In [ ]:
# Compile model
opt = optimizers.Adam()
top_model_ttr.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[categorical_accuracy, auc_roc])


In [ ]:
x_lr_train,y_lr_train= trn_features[:4000],  trn_labels[:4000]
x_lr_train.shape

In [ ]:
# model is a Keras model
lr_finder_top_model_ttr = LRFinder(top_model_ttr)

lr_finder_top_model_ttr.find(x_lr_train, y_lr_train, 0.000001, 1, batch_size=200, epochs=4)

lr_finder_top_model_ttr.plot_loss(n_skip_beginning=1, n_skip_end=1)

In [ ]:
import clr_callback
def get_callbacks_clr(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=10, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(name_weights, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    clr = clr_callback.CyclicLR(base_lr=0.0001, max_lr=0.01,
                        step_size=num_train_samples/train_batch_size_1/2)
    csv_logger = CSVLogger(filename = base_path + "\\model_history_{}_toplayer.csv".format(model_name), append=True)
    return [clr, earlystopping, ReduceLR, checkpoint, csv_logger]

def get_callbacks(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=15, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(name_weights, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    csv_logger = CSVLogger(filename = base_path + "\\model_history_{}_ALL.csv".format(modelname), append=True)
    return [earlystopping, ReduceLR, checkpoint, csv_logger]

name_weights = (base_path + "\\{}_toplayer.h5".format(model_name))
callbacks_list = get_callbacks_clr(name_weights = name_weights)

In [ ]:
# Fit model
#with tf.device('/device:CPU:0'):
history_top_model_ttr=top_model_ttr.fit(trn_features, trn_labels, epochs=50, 
                                        batch_size=batch_size, validation_data=(val_features, val_labels),
                                        callbacks=callbacks_list)

In [ ]:
top_model_ttr.save_weights(model_path+'top_model_ttr_vgg.h5')

In [ ]:
train_gen_1 = datagen_train.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size_1,
                                        class_mode='categorical')


val_gen = datagen_val.flow_from_directory(val_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

nb_train_samples = len(train_gen_1)
nb_validation_samples = len(val_gen)

In [ ]:
for layer in top_model_ttr.layers: 
    layer.trainable = True

In [ ]:
def create_model_VGG(model_input):
    base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_tensor=model_input, input_shape=input_shape)
    
    for layer in base_model_VGG.layers:
        layer.trainable = False
        print("trainable:", layer.name)
    
    x = base_model_VGG.output
    x = GlobalMaxPooling2D(name='1')(x)
    x = BatchNormalization(name='5')(out)
    x = Dropout(0.5, name='6')(x)
    x = Dense(512, activation='relu',name='7') (x)
    x = BatchNormalization(name='8')(x)
    x = Dropout(0.2, name='9')(x)
    x = Dense(256, activation='relu',name='10') (x)
    x = BatchNormalization(name='11')(x)
    
    predictions = (Dense(2, activation='softmax'))(x)
    model_VGG = Model(inputs=inputs, outputs=predictions)
    
    return top_model_tr

In [ ]:
model_VGG = create_model_VGG(model_input)

In [ ]:
model_xception.load_weights(model_path+'top_model_ttr_vgg.h5',by_name=True) 

In [ ]:
model_VGG.summary()

In [ ]:
# Compile model
opt = optimizers.Adam()

model_VGG.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[categorical_accuracy])

In [ ]:
import sys
sys.path.append(r"C:\Users\eg38emed\FCD\FCD vs TSC\keras_utils")
from keras_lr_finder import LRFinder

lr_gen = datagen_train.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=5000,
                                        class_mode='categorical')

x_lr_train,y_lr_train= lr_gen.next();x_lr_train.shape
lr_finder = LRFinder(model_VGG)

In [ ]:
lr_finder.find(x_lr_train, y_lr_train, 0.000001, 0.1, batch_size=64, epochs=4)
lr_finder.plot_loss(n_skip_beginning=1, n_skip_end=1)

In [ ]:
import clr_callback
def get_callbacks_clr(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=10, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(name_weights, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    clr = clr_callback.CyclicLR(base_lr=0.0001, max_lr=0.01,
                        step_size=num_train_samples/train_batch_size_1/2)
    csv_logger = CSVLogger(filename = base_path + "\\model_history_{}_toplayer.csv".format(model_name), append=True)
    return [clr, earlystopping, ReduceLR, checkpoint, csv_logger]

def get_callbacks(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=15, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(name_weights, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    csv_logger = CSVLogger(filename = base_path + "\\model_history_{}_ALL.csv".format(modelname), append=True)
    return [earlystopping, ReduceLR, checkpoint, csv_logger]

name_weights = (base_path + "\\{}_toplayer.h5".format(model_name))
callbacks_list = get_callbacks_clr(name_weights = name_weights)

In [ ]:
def create_model_VGG(model_input):
    base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_tensor=model_input, input_shape=input_shape)
    
    for layer in base_model_VGG.layers:
        layer.trainable = True
        print("trainable:", layer.name)
    
    x = base_model_VGG.output
    x = GlobalMaxPooling2D(name='1')(x)
    x = BatchNormalization(name='5')(out)
    x = Dropout(0.5, name='6')(x)
    x = Dense(512, activation='relu',name='7') (x)
    x = BatchNormalization(name='8')(x)
    x = Dropout(0.2, name='9')(x)
    x = Dense(256, activation='relu',name='10') (x)
    x = BatchNormalization(name='11')(x)
    
    predictions = (Dense(2, activation='softmax'))(x)
    model_VGG = Model(inputs=inputs, outputs=predictions)
    
    return top_model_tr